In [1]:
# JSSP library
#   includes: JSSP reader, heuristic skeletons etc
%run JSSP_skeleton.ipynb

#Includes solver for runing JSSP
%run Solver.ipynb

# Heuristics
%run heuristics.ipynb

# JSS Problem Generator
%run problemGen.ipynb

# File reader
%run fileReader.ipynb

# Normal Tests (beggining ML techniques)
%run firstTests.ipynb

#For saving data
%run dataSaving.ipynb


C:\Users\marti\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
def test_normalize_values():
    #NOTA: recuerda que se normaliza hacia "abajo"
    test_arr = [[1,1,1,1,2],[2,2,2,1,3],[2,3,2,1,2],[3,4,3,1,1]]
    normalize_values(test_arr, [0,1,3,4])
    print(test_arr)
test_normalize_values()  

[[0.0, 0.0, 1, 0, 0.5], [0.5, 0.3333333333333333, 2, 0, 1.0], [0.5, 0.6666666666666666, 2, 0, 0.5], [1.0, 1.0, 3, 0, 0.0]]


In [7]:
problems = [[4,4, 0,0]]

data_pre = {}
data_post = {}
pg = problemGenerator();
for i in problems:
    JSSProblem(1, pg.generate_problem(i[0],i[1], i[2],i[3]), test_solver(1), data_acts= data_pre, data_state = data_post)

[0, 0, 1, 0][0, 1, 1, 1][0, 2, 1, 2][0, 3, 1, 3]
[1, 0, 1, 0][1, 1, 1, 1][1, 2, 1, 2][1, 3, 1, 3]
[2, 0, 1, 0][2, 1, 1, 1][2, 2, 1, 2][2, 3, 1, 3]
[3, 0, 1, 0][3, 1, 1, 1][3, 2, 1, 2][3, 3, 1, 3]





0 para salir, 1 para nuevo movimiento, 2 para p, 3 para screen clear, 4 pre_acts, 5 post_acts, 6 pre_mach y 7 post_mach
 1
id job: 0
id activity: 0
id machine: 0
position: 0
[0, 1, 1, 1][0, 2, 1, 2][0, 3, 1, 3]
[1, 0, 1, 0][1, 1, 1, 1][1, 2, 1, 2][1, 3, 1, 3]
[2, 0, 1, 0][2, 1, 1, 1][2, 2, 1, 2][2, 3, 1, 3]
[3, 0, 1, 0][3, 1, 1, 1][3, 2, 1, 2][3, 3, 1, 3]

[0, 0, 1]



0 para salir, 1 para nuevo movimiento, 2 para p, 3 para screen clear, 4 pre_acts, 5 post_acts, 6 pre_mach y 7 post_mach
 0


In [4]:
#test of getBestAnswer function from ML solvers
def test_getBestAnswer():
    #To test the best answer train a tree with the following array (sum>3 is true)
    train_arr = [[1,1],[1,2],[2,2],[2,1],[1,0], [0,1],[0,2], [2,0]]
    train_labels = [False, True, True, True, False, False, False, False]

    def getBestAnswer(ml_alg, data, answers):
        best_answer = (None, -1)
        for i in range(len(data)):
            curr_state_val = ml_alg(data[i])
            print(curr_state_val, answers[i])
            if curr_state_val > best_answer[1]:
                best_answer = (answers[i], curr_state_val)                
        #print(best_answer[0])   
        return best_answer[0]

    def cross_validate_tests(X, Y, clf, n_splits = 4):

        kf = KFold(n_splits = n_splits)
        best = None
        bestAcc = 0
        scores = []
        X = np.array(X)
        Y = np.array(Y)
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            clf = clf.fit(X_train, Y_train)
            score = clf.score(X_test, Y_test)
            if score>bestAcc:
                best = clf
            ml_alg = lambda x: clf.predict([x])
            print(".", train_index, score)
            #After training use test data as data and answers
            test_data = [[1,1],[0,0],[2,1],[1,-1]]#returns [2,1]
            print(",",getBestAnswer(ml_alg, test_data, test_data))
            test_data = [[1,1],[0,0],[2,0],[1,2]] #returns [1,2]
            print(",",getBestAnswer(ml_alg, test_data, test_data))
            test_data = [[1,1],[0,0],[2,1],[1,2]] #returns [2,1]
            print(",",getBestAnswer(ml_alg, test_data, test_data))
            clf = clone(clf)
            scores.append(score)

        scores = np.array(scores)
        print(scores.mean(), scores.std())

        return scores, best


    kernel ="rbf"
    randomS = 1234
    print("using SVC")
    clf = make_pipeline(StandardScaler(), SVC(kernel = kernel, gamma='auto', random_state=randomS))

    scores, clf = cross_validate_tests(train_arr, train_labels, clf)


    #clf = ....
    ml_alg = lambda x: clf.predict([x])

    #After training use test data as data and answers
    test_data = [[1,1],[0,0],[2,1],[1,-1]]#returns [2,1]
    print(",",getBestAnswer(ml_alg, test_data, test_data))
    test_data = [[1,1],[0,0],[2,0],[1,2]] #returns [1,2]
    print(",",getBestAnswer(ml_alg, test_data, test_data))
    test_data = [[1,1],[0,0],[2,1],[1,2]] #returns [2,1]
    print(",",getBestAnswer(ml_alg, test_data, test_data))


test_getBestAnswer()

using SVC
. [2 3 4 5 6 7] 0.5
[False] [1, 1]
[False] [0, 0]
[ True] [2, 1]
[False] [1, -1]
, [2, 1]
[False] [1, 1]
[False] [0, 0]
[False] [2, 0]
[False] [1, 2]
, [1, 1]
[False] [1, 1]
[False] [0, 0]
[ True] [2, 1]
[False] [1, 2]
, [2, 1]
. [0 1 4 5 6 7] 0.0
[False] [1, 1]
[False] [0, 0]
[False] [2, 1]
[False] [1, -1]
, [1, 1]
[False] [1, 1]
[False] [0, 0]
[False] [2, 0]
[False] [1, 2]
, [1, 1]
[False] [1, 1]
[False] [0, 0]
[False] [2, 1]
[False] [1, 2]
, [1, 1]
. [0 1 2 3 6 7] 1.0
[False] [1, 1]
[False] [0, 0]
[ True] [2, 1]
[False] [1, -1]
, [2, 1]
[False] [1, 1]
[False] [0, 0]
[False] [2, 0]
[ True] [1, 2]
, [1, 2]
[False] [1, 1]
[False] [0, 0]
[ True] [2, 1]
[ True] [1, 2]
, [2, 1]
. [0 1 2 3 4 5] 1.0
[False] [1, 1]
[False] [0, 0]
[ True] [2, 1]
[False] [1, -1]
, [2, 1]
[False] [1, 1]
[False] [0, 0]
[False] [2, 0]
[ True] [1, 2]
, [1, 2]
[False] [1, 1]
[False] [0, 0]
[ True] [2, 1]
[ True] [1, 2]
, [2, 1]
0.625 0.414578098794425
[False] [1, 1]
[False] [0, 0]
[ True] [2, 1]
[False] [